<a href="https://colab.research.google.com/github/MohsenJadidi/Erdos-Goes-Neural/blob/main/ModifiedCode/Erdos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

import sys
sys.path.append('/content/drive/My Drive/ISLR Course - Fall 2020/Project')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install torch_scatter
!pip install torch_sparse
!pip install torch_geometric


In [4]:
!apt-get install python3-dev graphviz libgraphviz-dev pkg-config
!pip install graphviz
!pip install pygraphviz

!pip install visdom
!pip install GPUtill==1.4.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
pkg-config is already the newest version (0.29.1-0ubuntu2).
graphviz is already the newest version (2.40.1-2).
libgraphviz-dev is already the newest version (2.40.1-2).
python3-dev is already the newest version (3.6.7-1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.
ERROR: Could not find a version that satisfies the requirement GPUtill==1.4.0 (from versions: none)
ERROR: No matching distribution found for GPUtill==1.4.0


In [5]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"#imports
#imports
import torch
import torch.nn.functional as F

from torch.nn import Linear


#from kernel.datasets import get_dataset
from itertools import product

import time


from torch import tensor
from torch.optim import Adam
from torch.optim import SGD



from sklearn.model_selection import StratifiedKFold
from torch_geometric.data import DataLoader, DenseDataLoader as DenseLoader



from math import ceil

from torch.nn import Linear

from torch.distributions import categorical
from torch.distributions import Bernoulli

import torch.nn

%matplotlib inline
from matplotlib import pyplot as plt

#import pygraphviz as pgv

from torch_geometric.utils import convert as cnv
from torch_geometric.utils import sparse as sp

from torch_geometric.data import Data

#import pygraphviz as pgv

from networkx.drawing.nx_agraph import graphviz_layout

import networkx as nx


from torch.utils.data.sampler import RandomSampler

from torch.nn.functional import gumbel_softmax


from torch.distributions import relaxed_categorical

#import myfuncs #-----------------

#from tensorboardX import SummaryWriter

from torch_geometric.nn.inits import uniform
from torch_geometric.nn.inits import glorot, zeros


from torch.nn import Parameter

from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing

from torch_geometric.utils import degree

from torch_geometric.nn import GINConv, GATConv, global_mean_pool, NNConv, GCNConv



from torch.nn import Parameter

from torch.nn import Sequential as Seq, Linear, ReLU, LeakyReLU
from torch_geometric.nn import MessagePassing

import torch
import torch.nn.functional as F
from torch.nn import Linear, Sequential, ReLU, BatchNorm1d as BN
from torch_geometric.nn import GINConv, global_mean_pool


from torch_geometric.data import Batch 


from torch_scatter import scatter_min, scatter_max, scatter_add, scatter_mean


from torch import autograd

from torch_geometric.utils import to_dense_batch, to_dense_adj


from torch_geometric.utils import softmax, add_self_loops, remove_self_loops, segregate_self_loops, remove_isolated_nodes, contains_isolated_nodes, add_remaining_self_loops


from torch_geometric.utils import dropout_adj, to_undirected, to_networkx
from torch_geometric.utils import is_undirected

from cut_utils import get_diracs

import scipy

import scipy.io

from matplotlib.lines import Line2D

from torch_geometric.utils.convert import from_scipy_sparse_matrix

# import GPUtil #-----------!!!!!????


from networkx.algorithms.approximation import max_clique


import pickle


from torch_geometric.nn import SplineConv, global_mean_pool, DataParallel

from torch_geometric.data import DataListLoader
from random import shuffle


from networkx.algorithms.approximation import max_clique
from networkx.algorithms import graph_clique_number

from networkx.algorithms import find_cliques

from torch_geometric.nn.norm import graph_size_norm

In [6]:
import visdom 
from visdom import Visdom 
import numpy as np
import matplotlib.pyplot as plt




def plot_grad_flow( named_parameters):
    ave_grads = []
    layers = []
    for n, p in named_parameters:
        if(p.requires_grad) and ("bias" not in n):
            layers.append(n)
            ave_grads.append(p.grad.abs().mean())
            
    plt.plot(ave_grads, alpha=0.3, color="b")
    plt.yscale('log')
    plt.xticks(range(0,len(ave_grads), 1), layers, rotation="vertical")
    plt.xlim(xmin=0, xmax=len(ave_grads))
    plt.xlabel("Layers")
    plt.ylabel("average gradient")
    plt.title("Gradient flow")
    plt.grid(True)
    

class VisdomLinePlotter(object):
    """Plots to Visdom"""
    def __init__(self, env_name='main'):
        self.viz = Visdom(port=8097)
        self.env = env_name
        self.plots = {}
    def plot(self, var_name, split_name, title_name, x, y):
        if title_name not in self.plots:
            self.plots[title_name] = self.viz.line(X=np.array([x,x]), Y=np.array([y,y]), env=self.env, opts=dict(
                legend=[split_name],
                title=title_name,
                xlabel='Epochs',
                ylabel=var_name
            ))
        else:
            self.viz.line(X=np.array([x]), Y=np.array([y]), env=self.env, win=self.plots[title_name], name=split_name, update = 'append')
    

    def histog(self,title_name,vals):
        if title_name not in self.plots:
            self.plots[title_name] = self.viz.histogram(X=vals,env=self.env,opts=dict(title=title_name,numbins=20))
        else:
            self.viz.histogram(X=vals,env=self.env,win=self.plots[title_name],opts=dict(title=title_name,numbins=20, update = 'replace'))
  
    def gradflow(self, model, title_name):
                    title_name = "Gradflow"
                    layers = []
                    ave_grads = []
                    for n, p in net.named_parameters():
                            if(p.requires_grad) and ("bias" not in n):
                                     layers.append(str(n))
                                     ave_grads.append(p.grad.abs().mean().cpu().numpy())
                    if title_name not in self.plots:
                        self.plots[title_name]= self.viz.line(X= list(range(len(layers))),Y= np.array(ave_grads),  env = self.env,  opts=dict(fillarea=True, title = title_name, xtick = True,
        xtickmin=0, xtickmax=len(layers), xtickvals = list(range(len(layers))), xtickstep=1/len(layers), xticklabels = layers) )    
                    else:
                        self.viz.line(X= list(range(len(layers))), Y=ave_grads, env = self.env ,win=self.plots[title_name], opts=dict(fillarea=True, title = title_name, xtick = True,
        xtickmin=0, xtickmax=len(layers), xtickvals = list(range(len(layers))), xtickstep=1/len(layers), xticklabels = layers, update="append"))

In [7]:
class GCNConv2(MessagePassing):
    r"""The graph convolutional operator from the `"Semi-supervised
    Classfication with Graph Convolutional Networks"
    <https://arxiv.org/abs/1609.02907>`_ paper

    .. math::
        \mathbf{X}^{\prime} = \mathbf{\hat{D}}^{-1/2} \mathbf{\hat{A}}
        \mathbf{\hat{D}}^{-1/2} \mathbf{X} \mathbf{\Theta},

    where :math:`\mathbf{\hat{A}} = \mathbf{A} + \mathbf{I}` denotes the
    adjacency matrix with inserted self-loops and
    :math:`\hat{D}_{ii} = \sum_{j=0} \hat{A}_{ij}` its diagonal degree matrix.

    Args:
        in_channels (int): Size of each input sample.
        out_channels (int): Size of each output sample.
        improved (bool, optional): If set to :obj:`True`, the layer computes
            :math:`\mathbf{\hat{A}}` as :math:`\mathbf{A} + 2\mathbf{I}`.
            (default: :obj:`False`)
        cached (bool, optional): If set to :obj:`True`, the layer will cache
            the computation of :math:`{\left(\mathbf{\hat{D}}^{-1/2}
            \mathbf{\hat{A}} \mathbf{\hat{D}}^{-1/2} \right)}`.
            (default: :obj:`False`)
        bias (bool, optional): If set to :obj:`False`, the layer will not learn
            an additive bias. (default: :obj:`True`)
    """

    def __init__(self,
                 in_channels,
                 out_channels,
                 improved=False,
                 cached=False,
                 bias=True):
        super(GCNConv2, self).__init__('add')

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.improved = improved
        self.cached = cached
        self.cached_result = None

        self.weight = Parameter(torch.Tensor(in_channels, out_channels))

        if bias:
            self.bias = Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

    def reset_parameters(self):
        glorot(self.weight)
        zeros(self.bias)
        self.cached_result = None

    @staticmethod
    def norm(edge_index, num_nodes, edge_weight, improved=False, dtype=None):
        if edge_weight is None:
            edge_weight = torch.ones((edge_index.size(1), ),
                                     dtype=dtype,
                                     device=edge_index.device)
        edge_weight = edge_weight.view(-1)
        assert edge_weight.size(0) == edge_index.size(1)

#         print("before: ")
#         print(edge_weight.shape)
#         print(edge_index.shape)
        
        #edge_index, edge_weight = remove_self_loops(edge_index, edge_weight)
        #edge_index = add_self_loops(edge_index, num_nodes)
#         loop_weight = torch.full((num_nodes, ),
#                                  1 if not improved else 2,
#                                  dtype=edge_weight.dtype,
#                                  device=edge_weight.device)
        #edge_weight = torch.cat([edge_weight, loop_weight], dim=0)
#         print("after: ")
#         print(edge_weight.shape)
#         print(edge_index.shape)
        row, col = edge_index
        deg = scatter_add(edge_weight, row, dim=0, dim_size=num_nodes)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0

        return edge_index, deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]

    def forward(self, x, edge_index, edge_weight=None):
        """"""
        x = torch.matmul(x, self.weight)

        if not self.cached or self.cached_result is None:
            edge_index, norm = self.norm(edge_index, x.size(0), edge_weight,
                                         self.improved, x.dtype)
            self.cached_result = edge_index, norm
        edge_index, norm = self.cached_result

        return self.propagate(edge_index, x=x, norm=norm)

    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j

    def update(self, aggr_out):
        if self.bias is not None:
            aggr_out = aggr_out + self.bias
        return aggr_out

    def __repr__(self):
        return '{}({}, {})'.format(self.__class__.__name__, self.in_channels,
                                   self.out_channels)

In [9]:
def propagate(x, edge_index):
        row, col = edge_index
        out = scatter_add(x[col], row, dim=0)
        return out

def get_mask(x, edge_index, hops):
        for k in range(hops):
            x = propagate(x, edge_index)
        mask = (x>0).float()
        return mask


def total_var(x, edge_index, batch, undirected = True):
        row, col = edge_index
        if undirected:
            tv = (torch.abs(x[row]-x[col])) * 0.5
        else:
            tv = (torch.abs(x[row]-x[col]))
        
        tv = scatter_add(tv, batch[row], dim=0)
        return  tv

In [10]:
#from pytorch_memlab import profile, mem_reporter
#@profile
class cliqueMPNN_hindsight_earlyGAT(torch.nn.Module):
    def __init__(self, dataset, num_layers, hidden1, hidden2, deltas, elasticity=0.01, num_iterations = 30):
        super(cliqueMPNN_hindsight_earlyGAT, self).__init__()
        self.hidden1 = hidden1
        self.hidden2 = hidden2
        self.momentum = 0.1
        #self.nns = Sequential(
                #Linear(2*hidden, hidden*hidden),
                #LeakyReLU(0.1))
        self.num_iterations = num_iterations
        self.convs = torch.nn.ModuleList()
        self.deltas = deltas
        self.numlayers = num_layers
        self.elasticity = elasticity
        self.heads = 8
        self.concat = True
        
        self.bns = torch.nn.ModuleList()
        for i in range(num_layers-1):
            self.bns.append(BN(self.heads*self.hidden1, momentum=self.momentum))

        
#         self.nns = torch.nn.ModuleList()
#         for i in range(num_layers-1):
#             self.nns.append(Sequential(
#                 Linear(2*hidden, hidden*hidden),
#                 LeakyReLU(0.1)))
        
        self.convs = torch.nn.ModuleList()        
        for i in range(num_layers - 1):
                #self.convs.append(GATAConv(self.heads*self.hidden1, self.hidden1, concat=self.concat ,heads=self.heads))
                self.convs.append(GINConv(Sequential(
            Linear( self.heads*self.hidden1,  self.heads*self.hidden1),
            ReLU(),
            Linear( self.heads*self.hidden1,  self.heads*self.hidden1),
            ReLU(),
            BN(self.heads*self.hidden1, momentum=self.momentum),
        ),train_eps=True))
                #self.convs.append(GCNConv2(self.heads*self.hidden1, self.heads*self.hidden1))
        self.bn1 = BN(self.heads*self.hidden1)
        #self.conv1 = GATAConv(self.hidden2, self.hidden1, concat=self.concat ,heads=self.heads)
       
        self.conv1 = GINConv(Sequential(Linear(self.hidden2,  self.heads*self.hidden1),
            ReLU(),
            Linear( self.heads*self.hidden1,  self.heads*self.hidden1),
            ReLU(),
            BN(self.heads*self.hidden1, momentum=self.momentum),
        ),train_eps=True)

        if self.concat:
            self.lin1 = Linear(self.heads*self.hidden1, self.hidden1)
        else:
            self.lin1 = Linear(self.hidden1, self.hidden1)

        #self.bn2 = BN(self.hidden1, momentum=self.momentum)
        self.lin2 = Linear(self.hidden1, 1)
        self.gnorm = graph_size_norm.GraphSizeNorm()

                    


    def reset_parameters(self):
        self.conv1.reset_parameters()
        #self.conv2.reset_parameters()
        
        for conv in self.convs:
            conv.reset_parameters()
            
        for bn in self.bns:
            bn.reset_parameters()
            
        self.bn1.reset_parameters()
        self.lin1.reset_parameters()
        #self.bn2.reset_parameters()
        self.lin2.reset_parameters()






    def forward(self, data, edge_dropout = None, penalty_coefficient = 0.25):
        x = data.x
        edge_index = data.edge_index
        batch = data.batch
        num_graphs = batch.max().item() + 1
        row, col = edge_index     
        total_num_edges = edge_index.shape[1]
        N_size = x.shape[0]

        
        if edge_dropout is not None:
            edge_index = dropout_adj(edge_index, edge_attr = (torch.ones(edge_index.shape[1], device=device)).long(), p = edge_dropout, force_undirected=True)[0]
            edge_index = add_remaining_self_loops(edge_index, num_nodes = batch.shape[0])[0]
                
        reduced_num_edges = edge_index.shape[1]
        current_edge_percentage = (reduced_num_edges/total_num_edges)
        no_loop_index,_ = remove_self_loops(edge_index)  
        no_loop_row, no_loop_col = no_loop_index

        xinit= x.clone()

        #x = torch.cat([x.unsqueeze(-1),torch.rand((x.shape[0],self.hidden2-1),device=device)], dim=1)
        #x = torch.rand((x.shape[0],self.hidden2),device=device)
        #
        x = x.unsqueeze(-1)
        #mask = get_mask(x,edge_index,1).to(x.dtype)
        
#         print("xbefore:",x.shape)

        mask = get_mask(x,edge_index,1).to(x.dtype)
        x = F.leaky_relu(self.conv1(x, edge_index))# +x
        x = x*mask

#         print("xshape:",x.shape)
#         print("mask: ", mask.shape)
        
        #x = F.leaky_relu(self.conv1(x, edge_index)) + x
        #x = self.conv1(x, edge_index)
        #print(x.shape)
        x = self.gnorm(x)
        x = self.bn1(x)
        

        
        #print(x.shape)
            
        for conv, bn in zip(self.convs, self.bns):
            if(x.dim()>1):
                x =  x+F.leaky_relu(conv(x, edge_index))
                mask = get_mask(mask,edge_index,1).to(x.dtype)
                x = x*mask
                x = self.gnorm(x)
                x = bn(x)


#         x = self.conv2(x, edge_index)
#         mask = get_mask(mask,edge_index,1).to(x.dtype)
#         x = x*mask
        xpostconvs = x.detach()
        #
        x = F.leaky_relu(self.lin1(x)) 
        x = x*mask
        #x = self.gnorm(x)
        #x = self.bn2(x)


        xpostlin1 = x.detach()
        #x = F.dropout(x, p=0.0, training=self.training)
        x = F.leaky_relu(self.lin2(x)) 
       # x = self.gnorm(x)

        x = x*mask


        #xprethresh = x.detach()

        #x = x/scatter_add(torch.abs(x), batch,0)[batch]

        
        #calculate min and max
        batch_max = scatter_max(x, batch, 0, dim_size= N_size)[0]
        batch_max = torch.index_select(batch_max, 0, batch)        
        batch_min = scatter_min(x, batch, 0, dim_size= N_size)[0]
        batch_min = torch.index_select(batch_min, 0, batch)

                
        x = (x-batch_min)/(batch_max+1e-6-batch_min)
        #x = x*mask + mask*1e-6
        

        probs=x
        
        #print(probs.shape)
        
        x2 = x.detach()              
        deg = degree(row).unsqueeze(-1) 
        totalvol = scatter_add(deg.detach()*torch.ones_like(x, device=device), batch, 0)+1e-6
        totalcard = scatter_add(torch.ones_like(x, device=device), batch, 0)+1e-6
        
                
        #volume within receptive field
        #recvol_hard = scatter_add(deg*mask.float(), batch, 0, dim_size = num_graphs)+1e-6 
        #reccard_hard = scatter_add(mask.float(), batch, 0, dim_size = num_graphs)+1e-6 
        
        #assert recvol_hard.mean()/totalvol.mean() <=1, "Something went wrong! Receptive field is larger than total volume."

        
        
        x2 =  ((x2 - torch.rand_like(x, device = device))>0).float()
        
        vol_1 = scatter_add(probs*deg, batch, 0)+1e-6
        card_1 = scatter_add(probs, batch,0)            
        #rec_field = scatter_add(mask, batch, 0)+1e-6
        set_size = scatter_add(x2, batch, 0)
#         tv_hard = total_var(x2, edge_index, batch)
        vol_hard = scatter_add(deg*x2, batch, 0, dim_size = batch.max().item()+1)+1e-6 
#         conduct_hard = tv_hard/vol_hard
            
#         rec_field_ratio = set_size/rec_field
#         rec_field_volratio = vol_hard/recvol_hard
        total_vol_ratio = vol_hard/totalvol
        
        #volume within receptive field
        #recvol_hard = scatter_add(deg*mask.float(), batch, 0, dim_size = num_graphs)+1e-6 
        #reccard_hard = scatter_add(mask.float(), batch, 0, dim_size = num_graphs)+1e-6 
        
        
        #calculating the terms for the expected distance between clique and graph
        pairwise_prodsums = torch.zeros(num_graphs, device = device)
        for graph in range(num_graphs):
            batch_graph = (batch==graph)
            pairwise_prodsums[graph] = (torch.conv1d(probs[batch_graph].unsqueeze(-1), probs[batch_graph].unsqueeze(-1))).sum()/2
        
        
        ##############CAREFUL
        self_sums = scatter_add((probs*probs), batch, 0, dim_size = num_graphs)/2.

        #self_sums = scatter_add((probs*probs), batch, 0, dim_size = num_graphs)/1.
        
        #expected_weight_G = scatter_add(probs[no_loop_row]*probs[no_loop_col], batch[no_loop_row], 0, dim_size = num_graphs)/2
        
        expected_weight_G = scatter_add(probs[no_loop_row]*probs[no_loop_col], batch[no_loop_row], 0, dim_size = num_graphs)/2.
        expected_clique_weight = (pairwise_prodsums.unsqueeze(-1) - self_sums)/1.
        expected_distance = (expected_clique_weight - expected_weight_G)
        
        
        #lambda_factors = (torch.rand((30,1), device=device))*penalty_coefficient-0.10
        
        
        #hindsight = torch.ones_like(lambda_factors)*expected_distance.unsqueeze(-1)*0.5 - lambda_factors*expected_weight_G.unsqueeze(-1)
        

       # print(expected_clique_weight.shape)
        #print(self_sums.shape)

       # expected_loss =  #torch.median(hindsight, 1)[0]
        #print(expected_loss.shape)
        
        max_set_weight = (scatter_add(torch.ones_like(x)[no_loop_row], batch[no_loop_row], 0, dim_size = num_graphs)/2).squeeze(-1)
        
        #print("how many:", (data.batch==0).sum())
        
        set_weight = (scatter_add(x2[no_loop_row]*x2[no_loop_col], batch[no_loop_row], 0, dim_size = num_graphs)/2)+1e-6
        clique_edges_hard = (set_size*(set_size-1)/2) +1e-6
        clique_dist_hard = set_weight/clique_edges_hard
        
        
#          print("cardinalities: ", cardinalities)
#         print("max_set_weight:", max_set_weight)
#        print("max possible weight: ", max_possible_weight)
        #print(max_set_weight.shape)
        #print(max_possible_weight.shape)
        cardinalities =  scatter_add(torch.ones_like(data.batch),data.batch,dim=0).float()
        max_possible_weight = (cardinalities*(cardinalities-1)/2)
#         diff = (max_possible_weight - max_set_weight) 
        penalty_coefficient =   (max_set_weight/max_possible_weight) * penalty_coefficient
        #penalty_coefficient =  penalty_coefficient*(clique_dist_hard/(1-clique_dist_hard))
                
        #penalty_coefficient[clique_dist_hard>0.95] = 1.

        #print("pen coeff:", penalty_coefficient)
#         print("Diff: ", diff)
#         print("penalties:", penalties.shape)
#         print("exp dist:", expected_distance.shape)
        
        #assert ((max_possible_weight<max_set_weight).sum())<=1e-6, "Invalid calculation"
        expected_ratio = expected_weight_G/expected_distance
        expected_ratio = expected_ratio.detach()
        #penalty_coefficient = expected_ratio*penalty_coefficient
       # print(expected_ratio.shape)
                               
        #expected_loss = (penalty_coefficient)*expected_distance*0.5*expected_weight_G - 0.5*expected_weight_G
        #expected_loss = expected_clique_weight/expected_weight_G
        expected_loss = (penalty_coefficient)*expected_distance*0.5 - 0.5*expected_weight_G
        #expected_loss = (penalty_coefficient)*expected_clique_weight*0.5- 1.*expected_weight_G
        
#         print("pen coeff: ", penalty_coefficient)
#         print("loss: ", expected_loss)
    
    
#         for iter_graph in range(num_graphs):
#             if clique_dist_hard[iter_graph]<0.5:
#                 print("problematic graph exp distance: ", expected_distance[iter_graph])
#                 print("problematic graph exp cardinality : ", card_1[iter_graph])

#                 print("problematic graph num nodes: ", totalcard[iter_graph])
                
        clique_check = ((clique_edges_hard != clique_edges_hard))
        
        
        setedge_check  = ((set_weight != set_weight))
        
        
        
        assert ((clique_dist_hard>=1.1).sum())<=1e-6, "Invalid set vol/clique vol ratio."
        
#        print(expected_loss.shape)
        
       # normalize = cardinalities.max()/cardinalities
        #print(f"normalize: {normalize}")
        loss = expected_loss#*normalize
        #loss = expected_loss*(1/(clique_dist_hard+0.1))


        retdict = {}
        
        retdict["output"] = [probs.squeeze(-1),"hist"]   #output
        #retdict["clique_check"] = [clique_edges_hard, "hist"]
        #retdict["set_weight_check"] = [set_weight, "hist"]
        #retdict["|Expected_vol - Target|"]= [targetcheck.squeeze(-1), "hist"] #absolute distance from targetvol
        #retdict["Expected_volume"] = [vol_1.mean(),"sequence"] #volume
        retdict["Expected_cardinality"] = [card_1.mean(),"sequence"]
        retdict["Expected_cardinality_hist"] = [card_1,"hist"]
        retdict["losses histogram"] = [loss.squeeze(-1),"hist"]
        retdict["Set sizes"] = [set_size.squeeze(-1),"hist"]
        retdict["volume_hard"] = [vol_hard.mean(),"aux"] #volume2
        retdict["cardinality_hard"] = [set_size[0],"sequence"] #volumeq
        retdict["Expected weight(G)"]= [expected_weight_G.mean(), "sequence"]
        retdict["Expected maximum weight"] = [expected_clique_weight.mean(),"sequence"]
        retdict["Expected distance"]= [expected_distance.mean(), "sequence"]
        #retdict["cut1"] = [tv.mean(),"sequence"] #cut1
        #retdict["cut_hard"] = [tv_hard.mean(),"sequence"] #cut1
        #retdict["Average cardinality ratio of receptive field "] = [rec_field_ratio.mean(),"sequence"] 
        #retdict["Recfield volume/Total volume"] = [recvol_hard.mean()/totalvol.mean(), "sequence"]
        #retdict["Average ratio of receptive field volume"]= [rec_field_volratio.mean(),'sequence']
        retdict["Currvol/Cliquevol"] = [clique_dist_hard.mean(),'sequence']
        retdict["Currvol/Cliquevol all graphs in batch"] = [clique_dist_hard.squeeze(-1),'hist']
        retdict["Average ratio of total volume"]= [total_vol_ratio.mean(),'sequence']
        #retdict["penalty coffs"] = [penalty_coefficient.squeeze(-1),"hist"]
        retdict["cardinalities"] = [cardinalities.squeeze(-1),"hist"]
        retdict["Current edge percentage"] = [torch.tensor(current_edge_percentage),'sequence']
        #retdict["Clique Weight hard"] = [clique_edges_hard[0], 'sequence']
        #retdict["Set weight"] = [set_weight[0], 'sequence']
        #retdict["mask"] = [mask, "aux"] #mask
        #retdict["xinit"] = [xinit,"hist"] #layer input diracs
        #retdict["xpostlin1"] = [xpostlin1.mean(1),"hist"] #after first linear layer
        #retdict["xprethresh"] = [xprethresh.mean(1),"hist"] #pre thresholding activations 195 x 1
        #retdict["xsoftbin"] = [xsoftbin.mean(1),"hist"] #soft binarized output
        #retdict["lossvol"] = [lossvol.mean(),"sequence"] #volume constraint
        #retdict["losscard"] = [losscard.mean(),"sequence"] #cardinality constraint
        retdict["loss"] = [loss.mean().squeeze(),"sequence"] #final loss

        return retdict
    
    def __repr__(self):
        return self.__class__.__name__

In [11]:
def predict(model, data_loader, recfield):
    model.train()
    avg_loss = 0
    avg_cliqdist = 0
    exp_cardinalities = torch.tensor(0)
    for data in data_loader:
        optimizer.zero_grad()
        data = data.to(device)
        #print("data  batchsum: ",(data.batch==1).sum())
        data = get_diracs(data, 1, sparse = True, effective_volume_range=0.15, receptive_field = recfield)
        data = data.to(device)
        #print("data prime batchsum: ",(data.batch==1).sum())
        retdict = model(data)
        avg_loss += retdict['loss'][0].item()/len(data_loader)
        avg_cliqdist += retdict["Currvol/Cliquevol"][0].item()/len(data_loader)
        exp_cardinalities = [retdict["Expected_cardinality_hist"][0]]
    #print("retdict: ", retdict["Expected_cardinality_hist"][0] )
    return avg_loss, avg_cliqdist, exp_cardinalities

In [22]:
datasets = ["TWITTER_SNAP", "COLLAB", "IMDB-BINARY"]
dataset_name = datasets[2]
#datasetname = "COLLAB_shuffle_1"
#datasetname = "TWITTER_SNAP"
#dataset_name = "IMDB-BINARY"

if dataset_name == "TWITTER_SNAP":
    stored_dataset = open('datasets/TWITTER_SNAP.p', 'rb')        
elif dataset_name == "COLLAB":
    stored_dataset = open('datasets/dataset_shuffle_1'+'.p', 'rb')
elif dataset_name == "IMDB-BINARY":
    stored_dataset = open('/content/drive/My Drive/ISLR Course - Fall 2020/Project/IMDB_BINARY.p', 'rb')

dataset = pickle.load(stored_dataset)

dataset_scale = 1
# total_samples = int(np.floor(len(dataset)*dataset_scale))
total_samples = int(np.floor(dataset.len()*dataset_scale))
# dataset = dataset[:total_samples]

# num_trainpoints = int(np.floor(0.6*len(dataset)))
# num_valpoints = int(np.floor(num_trainpoints/3))
# num_testpoints = len(dataset) - (num_trainpoints + num_valpoints)

num_trainpoints = int(np.floor(0.6*dataset.len()))
num_valpoints = int(np.floor(num_trainpoints/3))
num_testpoints = dataset.len() - (num_trainpoints + num_valpoints)

traindata= dataset[0:num_trainpoints]
# valdata = dataset[num_trainpoints:num_trainpoints + num_valpoints]
# testdata = dataset[num_trainpoints + num_valpoints:]

# batch_size = 32

# train_loader = DataLoader(traindata, batch_size, shuffle=True)
# test_loader = DataLoader(testdata, batch_size, shuffle=False)
# val_loader =  DataLoader(valdata, batch_size, shuffle=False)


# #set up random seeds 
# torch.manual_seed(1)
# np.random.seed(2)   
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False

AttributeError: ignored

In [1]:
dataset[0]

NameError: ignored